## Setup

In [1]:
# We need to import these modules to get started
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import brain_observatory_utilities.datasets.behavior.data_formatting as behavior_utils
import brain_observatory_utilities.datasets.electrophysiology.utilities as ephys_utils

# seaborn makes pretty plots & sets font sizes nicely
import seaborn as sns
sns.set_context('notebook', font_scale=1.5, rc={'lines.markeredgewidth': 2})

import matplotlib
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# magic functions for jupyter notebook plotting
%matplotlib inline

In [2]:
import platform
platstring = platform.platform()

if ('Darwin' in platstring) or ('macOS' in platstring):
    # macOS 
    data_root = "/Volumes/Brain2024/"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif ('amzn' in platstring):
    # then on CodeOcean
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2024/"

In [3]:
# import behavior projet cache class from SDK to be able to load the data
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorNeuropixelsProjectCache

cache = VisualBehaviorNeuropixelsProjectCache.from_local_cache(cache_dir=data_root, use_static_cache=True)

# if we needed to download the data we could have used the following line
# cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=data_root)  

In [4]:
# get the metadata tables
units_table = cache.get_unit_table()

channels_table = cache.get_channel_table()

probes_table = cache.get_probe_table()

behavior_sessions_table = cache.get_behavior_session_table()

ecephys_sessions_table = cache.get_ecephys_session_table()

In [5]:
# file paths
recording_type = 'ephys'
postproc_dir = '/root/capsule/postprocessed/'+recording_type+'/'
fig_dir = '/root/capsule/figures/'+recording_type+'/'

## Postprocess ephys sessions

In [6]:
imageset_overlap_sessions = ecephys_sessions_table[(ecephys_sessions_table.prior_exposures_to_image_set==0) & (ecephys_sessions_table.experience_level=='Novel')]
imageset_overlap_sessions.head(2)

,behavior_session_id,date_of_acquisition,equipment_name,session_type,mouse_id,genotype,sex,project_code,age_in_days,unit_count,...,channel_count,structure_acronyms,image_set,prior_exposures_to_image_set,session_number,experience_level,prior_exposures_to_omissions,file_id,abnormal_histology,abnormal_activity
ecephys_session_id,,,,,,,,,,,,,,,,,,,,,
1044594870,1044624428,2020-08-20 15:03:56.422000+00:00,NP.1,EPHYS_1_images_H_5uL_reward,524761,wt/wt,F,NeuropixelVisualBehavior,152,2103,...,1920,"['CA1', 'CA3', 'DG-mo', 'DG-po', 'DG-sg', 'HPF...",H,0,2,Novel,1,872,NaN,NaN
1048189115,1048221709,2020-09-03 14:16:57.913000+00:00,NP.1,EPHYS_1_images_H_3uL_reward,509808,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,M,NeuropixelVisualBehavior,264,1925,...,2304,"['APN', 'CA1', 'CA3', 'DG-mo', 'DG-po', 'DG-sg...",H,0,2,Novel,1,879,NaN,NaN


In [7]:
filename = recording_type+'_imageset_overlap_sessions'
imageset_overlap_sessions.to_pickle(postproc_dir+filename+'.pkl')

In [14]:
1052533639 in imageset_overlap_sessions.index

True

## Load data

In [9]:
# for session_id in imageset_overlap_sessions.index:
#     print(f'Loading session {session_id}...')
#     session = cache.get_ecephys_session(ecephys_session_id=session_id)

In [ ]:
# load neural data
spike_times = session.spike_times

# load behavioral data
all_stim_pres = session.stimulus_presentations
stim_pres = behavior_utils.get_annotated_stimulus_presentations(session) # only image presentations in stim block 0 - why?

In [ ]:
# get unit and channel data, sort the units by depth and filter for "good" units
units = session.get_units() # contains information about spike waveforms, isolation quality
channels = session.get_channels() # contains information about anatomical location

unit_channels = units.merge(channels, left_on='peak_channel_id', right_index=True) # associate anatomical information with each unit

# sort units by depth
unit_channels = unit_channels.sort_values('probe_vertical_position', ascending=False)

# filter units using QC metrics
good_unit_filter = ((unit_channels['snr']>1)&
                    (unit_channels['isi_violations']<.5)&
                    (unit_channels['firing_rate']>0.1)&
                    (unit_channels['amplitude_cutoff']<0.1)&
                    (unit_channels['presence_ratio']>.9)
                   )

good_units = unit_channels.loc[good_unit_filter]

# filter by brain region
area_of_interest = 'VIS'
area_units = good_units[good_units['structure_acronym'].str.contains(area_of_interest)]
# area_units = good_units[good_units['structure_acronym'] == area_of_interest]
num_units = len(area_units)
print(num_units)